In [1]:
import numpy as np
import pandas as pd
from re import sub
from time import time
import pickle

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras_tqdm import TQDMNotebookCallback as ktqdm
from keras.utils import normalize
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import TensorBoard
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from keras.initializers import RandomUniform, RandomNormal
from keras.layers.advanced_activations import LeakyReLU

from tensorflow.nn import relu, softmax

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

import scipy.stats as st

import seaborn as sns

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def print_dict(d):
    print("{:<15} {:<15} {:<10}".format('NeuralNetwork','MEAN','STD'))
    for k, v in d.items():
        print("{:<15} {:<15} {:<10}".format(k, round(v['MEAN'], 4), round(v['STD'], 4)))

In [3]:
df = pd.read_csv('fifa19.csv')

In [4]:
df.drop(columns=['Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Real Face', 'Preferred Foot',
                 'Body Type', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 
                 'Name', 'Nationality', 'Work Rate', 'Position'],inplace=True)

## Obiettivo: predire valore dei giocatori

Pre-processing: convertire value, wage e release clause da string a float

In [5]:
curs=["Release Clause", "Value", "Wage"]

for cur in curs:
    def curr2val(x):
        x = str(x).replace('€', '')
        if 'K' in x: x = float(str(x).replace('K', '')) * 1000
        else: x = float(str(x).replace('M', '')) * 1000000
        return x
    df[cur] = df[cur].apply(curr2val)
    

Individuare eventuali outlier nella colonna value

In [6]:
def detect_outlier(data, threshold = 3):
    outliers=[]
    mean = np.mean(data)
    std = np.std(data)
    
    for y in data:
        score= (y - mean) / std 
        if np.abs(score) > threshold:
            outliers.append(y)
    return outliers

In [7]:
min_out = min(detect_outlier(df['Value'], threshold = 2))

df = df[df['Value'] < min_out] 
df = df[df['Value'] > 0]

Conversione in interi per le altre label

In [8]:
cols=["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW","LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM", "LWB", "LDM","CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"]
for col in cols:
    df[col] = df[col].str[:-2]
    df[col] = df[col].astype(float)

In [9]:
df['Height'] = df['Height'].str.replace("'",'.')
df['Height'] = df['Height'].astype(float)

df['Weight'] = df['Weight'].str[:-3]
df['Weight'] = df['Weight'].astype(float)

In [10]:
labelencoder_X = LabelEncoder()

df['Club'] = labelencoder_X.fit_transform(df['Club'])

Mescolo le righe del dataset

In [11]:
df_flt = df.sample(frac=1, random_state=42)

train_slice = int(len(df_flt) * 0.8)

train = df_flt[:train_slice]
test = df_flt[train_slice:]

In [12]:
y_train = train.loc[:, ['Value']]
X_train = train.drop(columns='Value')

y_test = test.loc[:, ['Value']]
X_test = test.drop(columns='Value')

Sostiuisco eventuali valori nan con la media della colonna

In [13]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X_train)
X_full = imputer.transform(X_train)

imputer = imputer.fit(y_train)
y_full = imputer.transform(y_train)

imputer = imputer.fit(X_test)
X_test_full = imputer.transform(X_test)

imputer = imputer.fit(y_test)
y_test_full = imputer.transform(y_test)

Scalo i valori, sia per i caratteri che per il target

In [14]:
# scaler = RobustScaler()
# scaler = scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# X_train_scaled = preprocessing.scale(X_train)
scaler = StandardScaler().fit(X_full)
X_scaled = scaler.transform(X_full)

scaler_train = StandardScaler().fit(y_full)
y_scaled = scaler_train.transform(y_full) 
# X_train_scaled, X_test_scaled

scaler = StandardScaler().fit(X_test_full)
X_test_scaled = scaler.transform(X_test_full)

scaler_test = StandardScaler().fit(y_test_full)
y_test_scaled = scaler_test.transform(y_test_full) 

In [15]:
pca = PCA(n_components=10) 

pca.fit(X_scaled) 

X_pca = pca.transform(X_scaled)

In [16]:
data = {
    'X_scaled': X_scaled,
    'y_scaled': y_scaled,
    'X_test_scaled': X_test_scaled,
    'y_test_scaled': y_test_scaled,
    'y_test_full': y_test_full,
}

with open('dataset_corr_labels', 'wb') as f:
    pickle.dump(data, f)

In [17]:
def coeff_determination(y_test, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_test-y_pred ))
    SS_tot = K.sum(K.square( y_test - K.mean(y_test) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()))

In [18]:
ts_board = TensorBoard(log_dir='value_predictions_v3/{}'.format('kfold_' + str(time())))

In [19]:
def mean_abs_error(prediction, target):
    res = 0
    tot = 0
    for i in range(len(target)):
        if target[i][0] != 0 and not np.isnan(prediction[i][0]):
#             print(prediction[i][0], target[i][0])
            res += abs(prediction[i][0] - target[i][0])
            tot += 1
#     print(tot, len(target), len(prediction))
    return round(res / tot, 2)

In [20]:
def mean_abs_error(prediction, target):
    mean = np.mean([abs(t[0] - p[0]) for t, p in zip(target, prediction) if not np.isnan(p[0])])
    return round(mean, 2)

# Cross-validation
### K-Fold

In [21]:
outer_kfold = KFold(n_splits=3, random_state=42, shuffle=False)
inner_kfold = KFold(n_splits=5, random_state=19, shuffle=False )

In [22]:
def build_nn(input_layer, neurons):
    model = Sequential()

    model.add(Dense(neurons, input_dim = input_layer, activation='relu'))

    model.add(Dense(1, activation = 'linear'))
    
    return model

In [23]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
total_results = []

for fold, (train_index_outer, test_index_outer) in enumerate(outer_kfold.split(X_scaled)):
    
    print('OUTER FOLD {}'.format(fold))
    
    max_score = -1
    optimal_neurons = -1
    
    X_train_outer, X_test_outer = X_scaled[train_index_outer], X_scaled[test_index_outer]
    y_train_outer, y_test_outer = y_scaled[train_index_outer], y_scaled[test_index_outer]
    
#     inner_mean_scores = []

    neurons = range(10, 110, 10)
    
    for neuron in neurons:
        
        print('NEURONS {}'.format(neuron))
        
        inner_scores = []
        
        for train_index_inner, test_index_inner in inner_kfold.split(X_train_outer):
            X_train_inner, X_test_inner = X_train_outer[train_index_inner], X_train_outer[test_index_inner]
            y_train_inner, y_test_inner = y_train_outer[train_index_inner], y_train_outer[test_index_inner]

            nn = build_nn(X_train_inner.shape[1], neuron)
            nn.compile(optimizer=opt, loss='mse', metrics=[coeff_determination])
            nn.fit(X_train_inner, y_train_inner, batch_size=125, epochs=100, shuffle=True, verbose=1)

            res = nn.predict(X_test_inner)
            loss, acc = nn.evaluate(X_train_inner, y_train_inner, verbose=1)
            
            reversed_res = scaler_train.inverse_transform(res)
            reversed_label = scaler_train.inverse_transform(y_test_inner)
#             inner_scores.append(
#                 {'ACC': acc * 100, 'MAE': mean_abs_error(reversed_res, reversed_label), 'MAX': max(reversed_label), 'MEAN': round(np.mean(reversed_label), 4)}
#             )
            inner_scores.append(acc)
            
            print("loss: %.4f, acc: %.4f%%" % (loss, acc*100))
            
        current_score = np.mean(inner_scores)
        
        if max_score < current_score:
            max_score = current_score
            optimal_neurons = neuron
        
#         inner_mean_scores.append({'n_{}'.format(neuron): inner_scores})

    total_results.append({'fold_{}'.format(fold): (max_score, optimal_neurons)})
#     total_results.append({'fold_{}'.format(fold): inner_mean_scores})

OUTER FOLD 0
NEURONS 10
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
7428/7428 [==============================] - 0s 29us/step - loss: 1.0522 - coeff_determination: -0.0857
Epoch 2/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.4062 - coeff_determination: 0.5822
Epoch 3/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.2741 - coeff_determination: 0.7214
Epoch 4/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.2007 - coeff_determination: 0.7978
Epoch 5/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.1612 - coeff_determination: 0.8370
Epoch 6/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.1388 - coeff_determination: 0.8591
Epoch 7/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.1241 - coeff_determination: 0.8753
Epoch 8/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.1128 - coeff_determination: 0.8846
Epoch 9/100
74

7428/7428 [==============================] - 0s 9us/step - loss: 0.0289 - coeff_determination: 0.9714
Epoch 70/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0285 - coeff_determination: 0.9712
Epoch 71/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0282 - coeff_determination: 0.9719
Epoch 72/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0282 - coeff_determination: 0.9713
Epoch 73/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0277 - coeff_determination: 0.9722
Epoch 74/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0274 - coeff_determination: 0.9713
Epoch 75/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0269 - coeff_determination: 0.9730
Epoch 76/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0265 - coeff_determination: 0.9732
Epoch 77/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0264 - coeff

7428/7428 [==============================] - 0s 9us/step - loss: 0.0322 - coeff_determination: 0.9673
Epoch 40/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0319 - coeff_determination: 0.9685
Epoch 41/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0319 - coeff_determination: 0.9682
Epoch 42/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0310 - coeff_determination: 0.9692
Epoch 43/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0305 - coeff_determination: 0.9699
Epoch 44/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0298 - coeff_determination: 0.9699
Epoch 45/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0296 - coeff_determination: 0.9696
Epoch 46/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0291 - coeff_determination: 0.9705
Epoch 47/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0281 - coeff_

7428/7428 [==============================] - 0s 7us/step - loss: 0.0554 - coeff_determination: 0.9417
Epoch 10/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0543 - coeff_determination: 0.9428
Epoch 11/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0516 - coeff_determination: 0.9446
Epoch 12/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0505 - coeff_determination: 0.9479
Epoch 13/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0489 - coeff_determination: 0.9465
Epoch 14/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0483 - coeff_determination: 0.9498
Epoch 15/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0471 - coeff_determination: 0.9507
Epoch 16/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0457 - coeff_determination: 0.9509
Epoch 17/100
7428/7428 [==============================] - 0s 10us/step - loss: 0.0448 - coef

7428/7428 [==============================] - 0s 7us/step - loss: 0.0180 - coeff_determination: 0.9809
Epoch 81/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0170 - coeff_determination: 0.9814
Epoch 82/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0171 - coeff_determination: 0.9814
Epoch 83/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0171 - coeff_determination: 0.9817
Epoch 84/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0168 - coeff_determination: 0.9819
Epoch 85/100
7428/7428 [==============================] - 0s 7us/step - loss: 0.0168 - coeff_determination: 0.9822
Epoch 86/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0166 - coeff_determination: 0.9817
Epoch 87/100
7428/7428 [==============================] - 0s 8us/step - loss: 0.0169 - coeff_determination: 0.9819
Epoch 88/100
7428/7428 [==============================] - 0s 9us/step - loss: 0.0167 - coeff_

...

7428/7428 [==============================] - 3s 349us/step - loss: 0.4138 - coeff_determination: 0.6089
Epoch 2/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0931 - coeff_determination: 0.9079
Epoch 3/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0781 - coeff_determination: 0.9249
Epoch 4/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0706 - coeff_determination: 0.9305
Epoch 5/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0641 - coeff_determination: 0.9389
Epoch 6/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0594 - coeff_determination: 0.9430
Epoch 7/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0561 - coeff_determination: 0.9461
Epoch 8/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0530 - coeff_determination: 0.9489
Epoch 9/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0503 - coef

7428/7428 [==============================] - 0s 15us/step - loss: 0.0125 - coeff_determination: 0.9878
Epoch 72/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0119 - coeff_determination: 0.9884
Epoch 73/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0130 - coeff_determination: 0.9871
Epoch 74/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0134 - coeff_determination: 0.9866
Epoch 75/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0116 - coeff_determination: 0.9885
Epoch 76/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0120 - coeff_determination: 0.9881
Epoch 77/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0116 - coeff_determination: 0.9885
Epoch 78/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0114 - coeff_determination: 0.9890
Epoch 79/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0112

7428/7428 [==============================] - 0s 19us/step - loss: 0.0214 - coeff_determination: 0.9783
Epoch 42/100
7428/7428 [==============================] - 0s 22us/step - loss: 0.0221 - coeff_determination: 0.9775
Epoch 43/100
7428/7428 [==============================] - 0s 21us/step - loss: 0.0213 - coeff_determination: 0.9783
Epoch 44/100
7428/7428 [==============================] - 0s 17us/step - loss: 0.0203 - coeff_determination: 0.9791
Epoch 45/100
7428/7428 [==============================] - 0s 19us/step - loss: 0.0204 - coeff_determination: 0.9796
Epoch 46/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0198 - coeff_determination: 0.9797
Epoch 47/100
7428/7428 [==============================] - 0s 16us/step - loss: 0.0205 - coeff_determination: 0.9789
Epoch 48/100
7428/7428 [==============================] - 0s 17us/step - loss: 0.0208 - coeff_determination: 0.9786
Epoch 49/100
7428/7428 [==============================] - 0s 16us/step - loss: 0.0195

7428/7428 [==============================] - 0s 15us/step - loss: 0.0400 - coeff_determination: 0.9583
Epoch 12/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0375 - coeff_determination: 0.9606
Epoch 13/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0368 - coeff_determination: 0.9619
Epoch 14/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0357 - coeff_determination: 0.9631
Epoch 15/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0349 - coeff_determination: 0.9632
Epoch 16/100
7428/7428 [==============================] - 0s 17us/step - loss: 0.0323 - coeff_determination: 0.9673
Epoch 17/100
7428/7428 [==============================] - 0s 18us/step - loss: 0.0323 - coeff_determination: 0.9668
Epoch 18/100
7428/7428 [==============================] - 0s 16us/step - loss: 0.0313 - coeff_determination: 0.9668
Epoch 19/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0297

7428/7428 [==============================] - 0s 15us/step - loss: 0.0110 - coeff_determination: 0.9885
Epoch 82/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0108 - coeff_determination: 0.9885
Epoch 83/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0115 - coeff_determination: 0.9877
Epoch 84/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0118 - coeff_determination: 0.9877
Epoch 85/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0107 - coeff_determination: 0.9885
Epoch 86/100
7428/7428 [==============================] - 0s 15us/step - loss: 0.0105 - coeff_determination: 0.9884
Epoch 87/100
7125/7428 [===========================>..] - ETA: 0s - loss: 0.0105 - coeff_determination: 0.9888

In [ ]:
total_results

In [ ]:
nn = build_nn(X_scaled.shape[1], 200)
nn.compile(optimizer=opt, loss='mean_absolute_error', metrics=[coeff_determination])
nn.fit(X_scaled, y_scaled, batch_size=125, epochs=100, shuffle=True, verbose=1)

res = nn.predict(X_test_scaled)

scaler_test = StandardScaler().fit(y_test.values)
res = scaler_test.inverse_transform(res)

mean_abs_error(res, y_test.values), max(y_test.values), round(np.mean(y_test.values), 2), round(r2_score(y_test.values, res), 4)